In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


def generate_association_rules(transactions, min_support, min_confidence):
    # Convert the list of transactions into a boolean matrix using the TransactionEncoder class
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)

    # Convert the boolean matrix into a pandas DataFrame
    df = pd.DataFrame(te_ary, columns=te.columns_)

    # Get the frequent itemsets with a minimum support value
    frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)

    # Generate association rules with a minimum confidence value
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

    # Filter and sort the association rules based on their desired metrics
    filtered_rules = rules[(rules['lift'] > 1) & (rules['confidence'] > min_confidence)]
    sorted_rules = filtered_rules.sort_values(by=['lift', 'confidence'], ascending=False)

    return sorted_rules


def get_most_likely_bought_items(transactions, input_item, min_support, min_confidence):
    # Generate association rules
    association_rules = generate_association_rules(transactions, min_support, min_confidence)

    # Check if association_rules dataframe is empty
    if association_rules.empty:
        return "No association rules found with the given minimum support and confidence values."

    # Check if input_item is present in any of the transactions
    if input_item not in set.union(*map(set, transactions)):
        return "The input item is not present in any of the transactions."

    # Get the most likely bought items with the input_item as antecedent
    rules_antecedent = association_rules[association_rules['antecedents'].apply(lambda x: input_item in x)]
    rules_antecedent = rules_antecedent.sort_values(by=['lift', 'confidence'], ascending=False)

    # Get the most likely bought items with the input_item as consequent
    rules_consequent = association_rules[association_rules['consequents'].apply(lambda x: input_item in x)]
    rules_consequent = rules_consequent.sort_values(by=['lift', 'confidence'], ascending=False)

    # Merge the two dataframes to get the most likely bought items overall
    merged_rules = pd.concat([rules_antecedent, rules_consequent])

    # Get the top 10 most likely bought items
    top_items = merged_rules.head(10)['antecedents'].apply(lambda x: list(x)[0])

    return list(top_items)

In [2]:
df = pd.read_csv('/content/drive/MyDrive/grouped_items.csv')
df.head()

,encoded_order_id,item_name
0,0,"['Mimosa Package', 'Fire Pie', 'Breakfast Sand..."
1,1,"['Focaccia', 'Iced Tea', 'Bubbling Shrimp', 'H..."
2,2,"['sunset on damen', 'Margaritas For Everyone',..."
3,3,"['Well Tequila (Centenario Plata)', 'Fire Pie'..."
4,4,"[""Lyre's Amalfi Spritz N/A"", 'Focaccia', 'Gril..."


In [ ]:
transactions = []
x = []
for i in range(len(df)):
  itm = df.iat[i, 1]
  x = itm.split(',')
  transaction = []
  for i in x:
    j = i.replace("'", "")
    if j.startswith("["):
      j = j[1:]
    elif j.endswith("]"):
      j = j[:1]
    if j.startswith(" "):
      j = j[1:]
    transaction.append(j)
  transactions.append(transaction)
transactions

In [ ]:
new_transactions=[]
for i in transactions:
  transaction=[]
  new_transactions.append(i[:-1])
new_transactions

In [25]:
input_item = 'Focaccia'
min_support = 0.01
min_confidence = 0.5

likely_items = get_most_likely_bought_items(new_transactions, input_item, min_support, min_confidence)
temp = likely_items[0]
i=0
while temp == input_item:
  i +=1
  if i == len(likely_items):
    temp = likely_items[0]
    break
  temp = likely_items[i]
  if temp == '':
    temp = likely_items[i+1]
  else:
    pass
print(temp)

Pomodoro


In [27]:
item_value = "{'originalValue': 'focasia', 'resolvedValues': [], 'interpretedValue': 'focasia'}"
item_value.split("'")[3]

'focasia'

In [34]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


def generate_association_rules(input_item):
    df = pd.read_csv('/content/drive/MyDrive/grouped_items.csv')
    transactions = []
    x = []
    for i in range(len(df)):
        itm = df.iat[i, 1]
        x = itm.split(',')
        transaction = []
        for i in x:
            j = i.replace("'", "")
            if j.startswith("["):
                j = j[1:]
            elif j.endswith("]"):
                j = j[:1]
            if j.startswith(" "):
                j = j[1:]
            transaction.append(j)
        transactions.append(transaction)
    new_transactions=[]
    for i in transactions:
        transaction=[]
        new_transactions.append(i[:-1])
    te = TransactionEncoder()
    te_ary = te.fit(new_transactions).transform(new_transactions)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
    filtered_rules = rules[(rules['lift'] > 1) & (rules['confidence'] > 0.5)]
    sorted_rules = filtered_rules.sort_values(by=['lift', 'confidence'], ascending=False)
    if sorted_rules.empty:
        return "No association rules found with the given minimum support and confidence values."
    # Get the most likely bought items with the input_item as antecedent
    rules_antecedent = sorted_rules[sorted_rules['antecedents'].apply(lambda x: input_item in x)]
    rules_antecedent = rules_antecedent.sort_values(by=['lift', 'confidence'], ascending=False)

    # Get the most likely bought items with the input_item as consequent
    rules_consequent = sorted_rules[sorted_rules['consequents'].apply(lambda x: input_item in x)]
    rules_consequent = rules_consequent.sort_values(by=['lift', 'confidence'], ascending=False)
    merged_rules = pd.concat([rules_antecedent, rules_consequent])
    top_items = merged_rules.head(10)['antecedents'].apply(lambda x: list(x)[0])
    return list(top_items)


def get_most_likely_bought_item(input_item):
    likely_items = generate_association_rules(input_item)
    temp = likely_items[0]
    i = 0
    while temp == input_item:
        i += 1
        if i == len(likely_items):
            temp = likely_items[0]
            break
        temp = likely_items[i]
        if temp == '':
            temp = likely_items[i+1]
        else:
            pass
    return temp


In [35]:
chk_itm = 'Focaccia'
result = get_most_likely_bought_item(chk_itm)
result

'Pomodoro'